# BCI competition 2a

C:\Users\pipo_\Downloads\desc_2a

In [129]:
import mne
import numpy as np
from mne.datasets import eegbci
import matplotlib.pyplot as plt
from os import listdir
from mne.channels import make_standard_montage
from scipy import signal
from scipy.linalg import sqrtm, inv 
from sklearn.model_selection import train_test_split, GridSearchCV, KFold
from sklearn.utils import shuffle
from mne.decoding import CSP
from sklearn.svm import SVC
from sklearn.manifold import TSNE
import seaborn as sns
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.model_selection import ShuffleSplit,StratifiedKFold ,cross_val_score, cross_val_predict, KFold
from sklearn.metrics import classification_report,confusion_matrix
from sklearn.neighbors import KNeighborsClassifier
from pyriemann.utils.distance import distance_riemann
from scipy.linalg import logm, expm


target_data_0 = "A06"
calibrate_size = 120
alignmentMethod = "EA"

condition_wLTL= "EA"


In [130]:
def GetRawEDF(target_subjects="pipo"):
    EEG_data = {}

    if target_subjects == "all":
        target_subjects = ["A01","A02","A03","A04","A05","A06","A07","A08","A09"]

    path = "D:\\BCI_Competition_2a\\Training\\"
    list_dir = listdir(path)

    for i in range (0,len(target_subjects)):

        raw_gdf = mne.io.read_raw_gdf(path+list_dir[i],preload = False, verbose = False)

        EEG_data[target_subjects[i]] = {"Raw_data": raw_gdf.copy()}

    print(f"Successful to create Data of {target_subjects}")

    return EEG_data

In [131]:
Raw_data = GetRawEDF(target_subjects="all")

Successful to create Data of ['A01', 'A02', 'A03', 'A04', 'A05', 'A06', 'A07', 'A08', 'A09']


c:\Python311\Lib\contextlib.py:144: RuntimeWarning: Channel names are not unique, found duplicates for: {'EEG'}. Applying running numbers for duplicates.
  next(self.gen)
c:\Python311\Lib\contextlib.py:144: RuntimeWarning: Channel names are not unique, found duplicates for: {'EEG'}. Applying running numbers for duplicates.
  next(self.gen)
c:\Python311\Lib\contextlib.py:144: RuntimeWarning: Channel names are not unique, found duplicates for: {'EEG'}. Applying running numbers for duplicates.
  next(self.gen)
c:\Python311\Lib\contextlib.py:144: RuntimeWarning: Channel names are not unique, found duplicates for: {'EEG'}. Applying running numbers for duplicates.
  next(self.gen)
c:\Python311\Lib\contextlib.py:144: RuntimeWarning: Channel names are not unique, found duplicates for: {'EEG'}. Applying running numbers for duplicates.
  next(self.gen)
c:\Python311\Lib\contextlib.py:144: RuntimeWarning: Channel names are not unique, found duplicates for: {'EEG'}. Applying running numbers for dup

In [132]:
def butter_bandpass(lowcut,highcut,fs,order):
    nyq = 0.5*fs
    low = lowcut/nyq
    high = highcut/nyq
    b,a = signal.butter(order,[low,high],'bandpass')
    return b,a

def butter_bandpass_filter(data,lowcut = 6,highcut = 30, order = 4):
    b,a = butter_bandpass(lowcut,highcut,250,order)
    y = signal.filtfilt(b,a,data,axis=2)
    return y

def GetEpoch(EEG_data, tmin=-2.0, tmax=6.0, crop=(0,2),baseline = (-0.5,0.0), trial_removal_th = 100):

    EEG_epoch = {}

    for key_subs in EEG_data:
        raw_edf = EEG_data[key_subs]["Raw_data"]

        events, event_dict = mne.events_from_annotations(raw_edf)

        if key_subs == 'A04':
            event_dict =  {'769': 5,
            '770': 6,
            '772': 8,
            '771': 7}
            mapping = {5: 0, 6: 1, 8: 2, 7: 3}
            selected_events = events[np.isin(events[:, 2], [5, 6, 7, 8])]

        else:
            event_dict =  {'769': 7,
            '770': 8,
            '772': 10,
            '771': 9}
            mapping = {7: 0, 8: 1, 10: 2, 9: 3}
            selected_events = events[np.isin(events[:, 2], [7, 8, 9, 10])]

        Epochs = mne.Epochs(raw_edf, selected_events, 
            tmin= tmin,  
            tmax= tmax,    
            event_id=event_dict,
            preload = True,
            event_repeated='drop',
            baseline=baseline,
            verbose=False
            )
        
        selected_ch = ['EEG-Fz', 'EEG-Cz', 'EEG-C3', 'EEG-C4', 'EEG-Pz']
        
        EEG_epoch[key_subs] =  {"Raw_Epoch": Epochs.copy().pick(selected_ch).crop(tmin= crop[0], tmax= crop[1])}

        train_data = EEG_epoch[key_subs]['Raw_Epoch'].copy().get_data() * 10e5 

        labels = EEG_epoch[key_subs]["Raw_Epoch"].copy().events[:,-1]

        labels = np.vectorize(mapping.get)(labels)

        outlier_trial = []
        for ii in range(0,train_data.shape[0]):
            if train_data[ii].max() > trial_removal_th or train_data[ii].min() < -trial_removal_th:
                outlier_trial.append(ii)
                print(key_subs,train_data[ii].min(), ii)
                print(key_subs,train_data[ii].max(), ii)

        EEG_epoch[key_subs]['Raw_Epoch'] = np.delete(train_data, outlier_trial, axis = 0)
        EEG_epoch[key_subs]['label'] = np.delete(labels, outlier_trial)

        filtered_data = butter_bandpass_filter(EEG_epoch[key_subs]['Raw_Epoch'], lowcut= 6, highcut= 32)
        EEG_epoch[key_subs]['Raw_Epoch'] = filtered_data

    return EEG_epoch

In [133]:
EEG_Epochs = GetEpoch(Raw_data ,tmin= -1.0, tmax= 4.0, crop = (0,4) ,baseline= (-0.5,0.0), trial_removal_th = 150)

Used Annotations descriptions: ['1023', '1072', '276', '277', '32766', '768', '769', '770', '771', '772']
Used Annotations descriptions: ['1023', '1072', '276', '277', '32766', '768', '769', '770', '771', '772']


C:\Users\pipo_\AppData\Local\Temp\ipykernel_14924\1188733013.py:52: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  train_data = EEG_epoch[key_subs]['Raw_Epoch'].copy().get_data() * 10e5


Used Annotations descriptions: ['1023', '1072', '276', '277', '32766', '768', '769', '770', '771', '772']


C:\Users\pipo_\AppData\Local\Temp\ipykernel_14924\1188733013.py:52: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  train_data = EEG_epoch[key_subs]['Raw_Epoch'].copy().get_data() * 10e5


Used Annotations descriptions: ['1023', '1072', '32766', '768', '769', '770', '771', '772']


...

C:\Users\pipo_\AppData\Local\Temp\ipykernel_14924\1188733013.py:52: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  train_data = EEG_epoch[key_subs]['Raw_Epoch'].copy().get_data() * 10e5


Used Annotations descriptions: ['1023', '1072', '276', '277', '32766', '768', '769', '770', '771', '772']


C:\Users\pipo_\AppData\Local\Temp\ipykernel_14924\1188733013.py:52: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  train_data = EEG_epoch[key_subs]['Raw_Epoch'].copy().get_data() * 10e5


Used Annotations descriptions: ['1023', '1072', '276', '277', '32766', '768', '769', '770', '771', '772']


C:\Users\pipo_\AppData\Local\Temp\ipykernel_14924\1188733013.py:52: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  train_data = EEG_epoch[key_subs]['Raw_Epoch'].copy().get_data() * 10e5
C:\Users\pipo_\AppData\Local\Temp\ipykernel_14924\1188733013.py:52: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  train_data = EEG_epoch[key_subs]['Raw_Epoch'].copy().get_data() * 10e5


In [134]:
def GetConfusionMatrix(models, X_train, X_test, y_train, y_test):
    y_pred = models.predict(X_train)
    print("Classification TRAIN DATA \n=======================")
    print(classification_report(y_true= y_train, y_pred=y_pred))
    print("Confusion matrix \n=======================")
    print(confusion_matrix(y_true= y_train, y_pred=y_pred))

    y_pred = models.predict(X_test)
    print("Classification TEST DATA \n=======================")
    print(classification_report(y_true=y_test, y_pred=y_pred))
    print("Confusion matrix \n=======================")
    print(confusion_matrix(y_true=y_test, y_pred=y_pred))
    

label_target = EEG_Epochs[target_data_0]['label']
x_train, x_test, y_train, y_test = train_test_split(EEG_Epochs[target_data_0]['Raw_Epoch'], label_target, test_size=0.3, random_state = 42, stratify=label_target)

csp = CSP(n_components = 5, reg=None, log=None, rank= 'info')
csp.fit(x_train, y_train)   

x_train = csp.transform(x_train)
x_test = csp.transform(x_test)

lda = LinearDiscriminantAnalysis()
score = cross_val_score(lda, x_train, y_train, cv= 10)
print("LDA only Cross-validation scores:", np.mean(score))
lda.fit(x_train, y_train)

GetConfusionMatrix(lda, x_train, x_test, y_train, y_test)


Computing rank from data with rank='info'
    MAG: rank 5 after 0 projectors applied to 5 channels
Reducing data rank from 5 -> 5
Estimating covariance using EMPIRICAL
Done.


Computing rank from data with rank='info'
    MAG: rank 5 after 0 projectors applied to 5 channels
Reducing data rank from 5 -> 5
Estimating covariance using EMPIRICAL
Done.
Computing rank from data with rank='info'
    MAG: rank 5 after 0 projectors applied to 5 channels
Reducing data rank from 5 -> 5
Estimating covariance using EMPIRICAL
Done.
Computing rank from data with rank='info'
    MAG: rank 5 after 0 projectors applied to 5 channels
Reducing data rank from 5 -> 5
Estimating covariance using EMPIRICAL
Done.
LDA only Cross-validation scores: 0.37357142857142855
Classification TRAIN DATA 
              precision    recall  f1-score   support

           0       0.37      0.46      0.41        50
           1       0.45      0.43      0.44        51
           2       0.49      0.38      0.43        50
           3       0.36      0.36      0.36        50

    accuracy                           0.41       201
   macro avg       0.42      0.41      0.41       201
weighted avg     

In [135]:
from BCIAllFunction import BCIFuntions

calibrate_size = calibrate_size / EEG_Epochs[target_data_0]['Raw_Epoch'].shape[0]
AllBCIClass = BCIFuntions(numclass = 4, frequency = 250, ch_pick = ['Fz','C3', 'Cz','C4','Pz'])


if alignmentMethod == "LA":
    AllBCIClass.ComputeLA(EEG_Epochs, target_subject= target_data_0, calibrate_size=calibrate_size)
    if calibrate_size != 0:
        target_data = target_data_0 + "_test"
    count = 0

    for index in range(len(EEG_Epochs[target_data]['KMediod_label'])):
        if EEG_Epochs[target_data]['label'][index] == EEG_Epochs[target_data]['KMediod_label'][index]:
            count += 1
    print(count/len(EEG_Epochs[target_data]['KMediod_label']) * 100)
    
else:
    AllBCIClass.GetRawSet_ComputeEA(EEG_Epochs, target_subject= target_data_0, calibrate_size=calibrate_size)
    if calibrate_size != 0:
        target_data = target_data_0 + "_test"

In [136]:
EEG_Epochs['A01'].keys()

dict_keys(['Raw_Epoch', 'label', 'Raw_left', 'Raw_right', 'Raw_non', 'Raw_feet', 'EA_left', 'EA_right', 'EA_feet', 'EA_non', 'EA_Epoch'])

In [137]:
AllBCIClass.classifyCSP_LDA(EEG_Epochs, target_subjects= target_data, condition = "noEA") #target_sub is used for target_data otherwise are source_data

Computing rank from data with rank='info'
    MAG: rank 5 after 0 projectors applied to 5 channels
Reducing data rank from 5 -> 5
Estimating covariance using EMPIRICAL
Done.
Computing rank from data with rank='info'
    MAG: rank 5 after 0 projectors applied to 5 channels
Reducing data rank from 5 -> 5
Estimating covariance using EMPIRICAL
Done.
Computing rank from data with rank='info'
    MAG: rank 5 after 0 projectors applied to 5 channels
Reducing data rank from 5 -> 5
Estimating covariance using EMPIRICAL
Done.
Computing rank from data with rank='info'
    MAG: rank 5 after 0 projectors applied to 5 channels
Reducing data rank from 5 -> 5
Estimating covariance using EMPIRICAL
Done.
LDA only Cross-validation scores: 0.3878022650749923
Classification TRAIN DATA 
              precision    recall  f1-score   support

           0       0.38      0.47      0.42       606
           1       0.40      0.51      0.45       606
           2       0.40      0.29      0.33       606
       

In [138]:
AllBCIClass.classifyCSP_LDA(EEG_Epochs, target_subjects= target_data, condition = "EA") 

Computing rank from data with rank='info'
    MAG: rank 5 after 0 projectors applied to 5 channels
Reducing data rank from 5 -> 5
Estimating covariance using EMPIRICAL
Done.
Computing rank from data with rank='info'
    MAG: rank 5 after 0 projectors applied to 5 channels
Reducing data rank from 5 -> 5
Estimating covariance using EMPIRICAL
Done.
Computing rank from data with rank='info'
    MAG: rank 5 after 0 projectors applied to 5 channels
Reducing data rank from 5 -> 5
Estimating covariance using EMPIRICAL
Done.
Computing rank from data with rank='info'
    MAG: rank 5 after 0 projectors applied to 5 channels
Reducing data rank from 5 -> 5
Estimating covariance using EMPIRICAL
Done.
LDA only Cross-validation scores: 0.4740043532972826
Classification TRAIN DATA 
              precision    recall  f1-score   support

           0       0.51      0.57      0.54       606
           1       0.51      0.48      0.49       606
           2       0.50      0.54      0.52       606
       

# WLTL

In [139]:
CSP2D_Epoch = AllBCIClass.computeCSPFeatures(EEG_Epochs, target_subject = target_data)

Computing rank from data with rank='info'
    MAG: rank 5 after 0 projectors applied to 5 channels
Reducing data rank from 5 -> 5
Estimating covariance using EMPIRICAL
Done.
Computing rank from data with rank='info'
    MAG: rank 5 after 0 projectors applied to 5 channels
Reducing data rank from 5 -> 5
Estimating covariance using EMPIRICAL
Done.
Computing rank from data with rank='info'
    MAG: rank 5 after 0 projectors applied to 5 channels
Reducing data rank from 5 -> 5
Estimating covariance using EMPIRICAL
Done.
Computing rank from data with rank='info'
    MAG: rank 5 after 0 projectors applied to 5 channels
Reducing data rank from 5 -> 5
Estimating covariance using EMPIRICAL
Done.
Computing rank from data with rank='info'
    MAG: rank 5 after 0 projectors applied to 5 channels
Reducing data rank from 5 -> 5
Estimating covariance using EMPIRICAL
Done.
Computing rank from data with rank='info'
    MAG: rank 5 after 0 projectors applied to 5 channels
Reducing data rank from 5 -> 5


In [140]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import CategoricalCrossentropy

# Custom loss function
class CustomLossLL1(tf.keras.losses.Loss):
    def __init__(self, lambda_t, model):
        super().__init__()
        self.lambda_t = lambda_t
        self.cross_entropy = CategoricalCrossentropy()
        self.model = model

    def call(self, y_true, y_pred):
        ce_loss = self.cross_entropy(y_true, y_pred)
        ws = self.get_weights_from_model()
        reg_term = self.regularization_term(ws)
        return ce_loss + self.lambda_t * reg_term

    def get_weights_from_model(self):
        model_weights = []
        for layer in self.model.layers:
            if len(layer.get_weights()) > 0:
                model_weights.append(layer.get_weights()[0])
        # return tf.concat([tf.reshape(w, [-1]) for w in model_weights], axis=0)
        return model_weights

    def regularization_term(self, ws):
        reg_term = tf.pow(tf.norm(ws, ord='euclidean'),2)
        return reg_term


# Custom training loop
def custom_train_step(model, optimizer, x, y, custom_loss):
    with tf.GradientTape() as tape:
        y_pred = model(x, training=True) # Perform a forward pass and compute the predictions
        loss = custom_loss(y, y_pred) # Compute the custom loss
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    return loss


def train_weight_LL(X_train, y_train, lambd, num_tier=10, learning_rate = 0.01):
    n_classes = np.unique(y_train).size
    y_one_hot = tf.keras.utils.to_categorical(y_train, num_classes=n_classes)

    lambda_t = lambd  # Regularization parameter

    model = Sequential([
        Dense(n_classes, input_shape=(X_train.shape[1],), activation='softmax')  # Adjust input_shape to match the number of features in X
    ])

    # Compile the model
    optimizer = Adam(learning_rate)
    custom_loss = CustomLossLL1(lambda_t, model)

    # Custom training loop
    epochs = num_tier
    lowest_loss = float('inf')
    best_weights = None
    for epoch in range(epochs):
        loss = custom_train_step(model, optimizer, X_train, y_one_hot, custom_loss)
        loss_value = loss.numpy()
        if loss_value < lowest_loss:
            lowest_loss = loss_value
            best_weights = [layer.get_weights() for layer in model.layers]

        if epoch % 20 == 0:
            print(f"Epoch {epoch}, Loss: {loss.numpy()}")

    # best_weights = [layer.get_weights() for layer in model.layers]

    return best_weights, lowest_loss

def build_clf_params(data, target_subjects ,condition):

    for sub in data.keys():
        if (sub  != target_subjects) and  (sub != target_data_0): #Don't apply weight to target subject
            # Where the tranining data is stored
            if condition == "noEA":
                X = data[sub]['Raw_csp']
                y = data[sub]['Raw_csp_label']
                store_ws = 'ws_Raw'

            else:
                X = data[sub]['EA_csp']
                y = data[sub]['EA_csp_label']
                store_ws = 'ws_EA'

            weights, loss = train_weight_LL(X_train=X, y_train=y, lambd= 0.1, num_tier=1000, learning_rate= 0.005)
            print("weights of ", str(sub), ": ", weights)
            print("Lowest loss of ", str(sub), ": ", loss)
            data[sub][store_ws] = weights

build_clf_params(CSP2D_Epoch, target_subjects= target_data ,condition = condition_wLTL)

Epoch 0, Loss: 1.892957329750061
Epoch 20, Loss: 1.7133259773254395
Epoch 40, Loss: 1.6421326398849487
Epoch 60, Loss: 1.5849602222442627
Epoch 80, Loss: 1.5396479368209839
Epoch 100, Loss: 1.5084420442581177
Epoch 120, Loss: 1.4916439056396484
Epoch 140, Loss: 1.4888875484466553
Epoch 160, Loss: 1.4994882345199585
Epoch 180, Loss: 1.5225889682769775
Epoch 200, Loss: 1.557239055633545
Epoch 220, Loss: 1.602473497390747
Epoch 240, Loss: 1.6573600769042969
Epoch 260, Loss: 1.7210259437561035
Epoch 280, Loss: 1.7926750183105469
Epoch 300, Loss: 1.8715918064117432
Epoch 320, Loss: 1.957141399383545
Epoch 340, Loss: 2.0487654209136963
Epoch 360, Loss: 2.1459765434265137
Epoch 380, Loss: 2.248350143432617
Epoch 400, Loss: 2.355515480041504
Epoch 420, Loss: 2.467151165008545
Epoch 440, Loss: 2.5829758644104004
Epoch 460, Loss: 2.702742338180542
Epoch 480, Loss: 2.8262338638305664
Epoch 500, Loss: 2.9532551765441895
Epoch 520, Loss: 3.083634853363037
Epoch 540, Loss: 3.217214822769165
Epoch 56

In [141]:
# First define the kl divergence
def KL_div(P, Q):
    # First convert to np array
    P = np.array(P)
    Q = np.array(Q)
    
    # Then compute their means, datain shape of samples x feat
    mu_P = np.mean(P, axis=0)
    mu_Q = np.mean(Q, axis=0)    

    
    # Compute their covariance
    sigma_P = np.cov(P, rowvar=False)
    sigma_Q = np.cov(Q, rowvar=False)  

    diff = mu_Q - mu_P

    inv_sigma_Q = np.linalg.inv(sigma_Q)
    term1 = np.dot(np.dot(diff.T, inv_sigma_Q), diff)
    
    # Calculate the trace term trace(Sigma_Q^{-1} * Sigma_P)
    term2 = np.trace(np.dot(inv_sigma_Q, sigma_P))
    
    # Calculate the determinant term ln(det(Sigma_P) / det(Sigma_Q))
    det_sigma0 = np.linalg.det(sigma_P)
    det_sigma1 = np.linalg.det(sigma_Q)

    
    epsilon = 1e-6
    term3 = np.log((det_sigma0+epsilon) / (det_sigma1+epsilon))
    
    print(term3)
    
    # Dimensionality of the data
    K = mu_P.shape[0]
    
    # KL divergence
    kl_div = 0.5 * (term1 + term2 - term3 - K)
    
    return kl_div

In [142]:
# Compute kl divergence of target subject to each source subject
def compute_all_kl_div(data, target_subjects , condition):
    '''
    Parameter:
    data, is the whole data containing target and source data
    '''
    kl_div_score = []

    if condition == "noEA":
        target_data = 'Raw_csp'
        label_name = 'Raw_csp_label'

    else:
        target_data = 'EA_csp'
        label_name = 'EA_csp_label'
        
    # cal P from target data
    label_tgt =  data[target_subjects][label_name]
    P_left =  data[target_subjects][target_data][np.where(label_tgt == 0)]
    P_right = data[target_subjects][target_data][np.where(label_tgt == 1)]
    P_non = data[target_subjects][target_data][np.where(label_tgt == 2)]
    P_feet = data[target_subjects][target_data][np.where(label_tgt == 3)]

    tgt_data = target_subjects + "_test"

    #cal Q from each source subject
    for sub in data.keys():
        if (sub != target_subjects) and (sub != tgt_data):
            label_src =  data[sub][label_name]
            Q_left =  data[sub][target_data][np.where(label_src == 0)]
            Q_right = data[sub][target_data][np.where(label_src == 1)]
            Q_non = data[sub][target_data][np.where(label_src == 2)]
            Q_feet = data[sub][target_data][np.where(label_src == 3)]

            kl_left = KL_div(P_left, Q_left)
            kl_right = KL_div(P_right, Q_right)
            kl_non = KL_div(P_non, Q_non)
            kl_feet = KL_div(P_feet, Q_feet)

            # kl_div = (kl_left + kl_right+ kl_non + kl_feet)/4

            kl_div_temp = [kl_left, kl_right, kl_non, kl_feet]

            kl_div_score.append(kl_div_temp)

    data[target_subjects]['kl_div'] = kl_div_score


compute_all_kl_div(CSP2D_Epoch, target_subjects=target_data_0 ,condition = condition_wLTL) #target_sub for cal KL is calibrate set

1.2892522866877802
0.05778548245622659
0.3926780743664433
0.24505163115428116
1.2848244695027986
0.04431032919710483
0.3856292491889666
0.26316216040562906
1.1926898355483762
-0.049909494344152525
0.3411886338992855
0.13360733501237046
1.2877201694764768
0.03290675367735027
0.39838635976921644
0.23620018618345692
1.2241706568707633
-0.0597029223176751
0.3180565662575529
0.06747849473863131
1.3046792075410272
0.04622015135688045
0.39826033362990176
0.25616323212059744
1.2339710594846274
0.027608500205255973
0.3420750465000283
0.19390946200963297
0.48762253693713664
-0.38487243535726917
0.27448644508107145
-0.3203919916575563


In [143]:
np.array(CSP2D_Epoch[target_data_0]['kl_div'])

array([[ 5.49981156,  4.99347434,  8.24152365,  3.63779696],
       [ 5.32767393,  1.52156024,  3.52612037,  4.59705701],
       [ 4.18652534,  1.93403677,  3.28925458,  2.26193964],
       [ 4.41365757,  1.14857994,  3.83123545,  2.80574815],
       [ 3.21629018,  0.69981306,  2.18983865,  1.65530914],
       [ 7.32939934,  2.0707374 ,  4.30802669,  5.05504452],
       [ 3.83900779,  1.32869118,  3.5925581 ,  2.76258951],
       [ 0.63642639, -0.12422276,  2.53530803,  1.2867409 ]])

In [144]:
def compute_similarity_weights(data, target_subjects):
    kl = data[target_subjects]['kl_div']
    KL_inv_left = []
    KL_inv_right = []
    KL_inv_non = []
    KL_inv_feet = []

    alpha_s = []
    eps = 0.0001
    
    #equation (9)
    for val in kl:
        if val != 0: 
            KL_inv_left.append(1/((val[0] + eps)**4))
            KL_inv_right.append(1/((val[1] + eps)**4))
            KL_inv_non.append(1/((val[2] + eps)**4))
            KL_inv_feet.append(1/((val[3] + eps)**4))

    print(KL_inv_left)
    print(KL_inv_right)
    
    for i in range(0,len(KL_inv_left)):
        temp = [KL_inv_left[i]/sum(KL_inv_left), KL_inv_right[i]/sum(KL_inv_right), KL_inv_non[i]/sum(KL_inv_non), KL_inv_feet[i]/sum(KL_inv_feet)]
        alpha_s.append(temp)

    print(np.array(alpha_s))
                
    data[target_subjects]['alpha_s'] = alpha_s

compute_similarity_weights(CSP2D_Epoch, target_subjects=target_data_0)

[0.0010928918246028447, 0.0012411287777504341, 0.0032549446544421416, 0.0026349119038919524, 0.009343833514665369, 0.000346499617420143, 0.004603400644880859, 6.091641629339231]
[0.0016082513323604115, 0.1865216416611863, 0.07145806805388064, 0.5743860049155737, 4.167001043407393, 0.05437709463402768, 0.32075471527696975, 4213.027555134919]
[[1.78747687e-04 3.81246429e-07 2.24720475e-03 9.86427686e-03]
 [2.02992550e-04 4.42161672e-05 6.70588284e-02 3.86820500e-03]
 [5.32361773e-04 1.69395994e-05 8.85627292e-02 6.59881539e-02]
 [4.30952450e-04 1.36161935e-04 4.81163914e-02 2.78746980e-02]
 [1.52822868e-03 9.87814675e-04 4.50781380e-01 2.30062289e-01]
 [5.66716704e-05 1.28904436e-05 3.00981563e-02 2.64565081e-03]
 [7.52908203e-04 7.60369896e-05 6.22343567e-02 2.96577713e-02]
 [9.96317137e-01 9.98725559e-01 2.50900953e-01 6.30038955e-01]]


In [145]:
def compute_ETL_and_mu_ws(data, target_subjects, condition):

    mu_ws = 0
    temp_ws = 0

    if condition == "noEA":
        ws_name = 'ws_Raw'
    else:
        ws_name = 'ws_EA'

    alpha_s = np.array(data[target_subjects]['alpha_s'])

    tgt_data = target_subjects + "_test"
    index_count = 0

    for sub in data.keys():
        if (sub != target_subjects) and (sub != tgt_data):
            ws = data[sub][ws_name][0][0]
            # mu_ws += ws @ alpha_s  #equation (10)
            # mu_ws += np.dot(ws, np.transpose(alpha_s))
            mu_ws += ws * alpha_s[index_count]
            index_count += 1

    print(np.array(mu_ws))

    index_count = 0
    for sub in data.keys():
        if (sub != target_subjects) and (sub != tgt_data):
            ws = data[sub][ws_name][0][0]
            # ws_min_mu = np.dot((np.dot(ws,np.transpose(alpha_s)) - mu_ws) , np.transpose((np.dot(ws,np.transpose(alpha_s)) - mu_ws)))
            ws_min_mu = np.dot(((ws * alpha_s[index_count]) - mu_ws), np.transpose((ws * alpha_s[index_count]) - mu_ws))
            temp_ws += ws_min_mu #equation (11)
            index_count += 1

    print(np.array(temp_ws))
    
    # den = np.diag(temp_ws) #get array in diagonal line

    den = temp_ws
    nom = np.trace(temp_ws) #Return the sum along diagonals of the array.
    Sigma_TL = den/nom


    data[target_subjects]['Sigma_TL'] = Sigma_TL
    data[target_subjects]['mu_ws'] = mu_ws

compute_ETL_and_mu_ws(CSP2D_Epoch, target_subjects = target_data_0, condition=condition_wLTL)

[[-0.25891066 -0.33555171  0.13691678  0.04893298]
 [-0.65236331 -0.2853304  -0.24002355  0.15529069]
 [ 0.32806539  0.68077886  0.22829113 -0.08177716]
 [-0.45099622 -1.07746185 -0.01710004 -0.03701628]
 [ 0.08807441 -0.13505119 -0.01358355 -0.35822562]]
[[ 1.42991548e+00  1.67990036e+00 -2.03714417e+00  3.33151916e+00
  -8.39628582e-03]
 [ 1.67990036e+00  4.11215369e+00 -3.30378126e+00  4.19314689e+00
  -5.12454972e-01]
 [-2.03714417e+00 -3.30378126e+00  4.39376903e+00 -6.17636576e+00
  -2.23165618e-01]
 [ 3.33151916e+00  4.19314689e+00 -6.17636576e+00  9.57119659e+00
   8.10555002e-01]
 [-8.39628582e-03 -5.12454972e-01 -2.23165618e-01  8.10555002e-01
   1.17328969e+00]]


In [146]:
print(np.array(CSP2D_Epoch[target_data_0]['Sigma_TL']).shape)
print(np.array(CSP2D_Epoch[target_data_0]['mu_ws']).shape)

(5, 5)
(5, 4)


In [147]:
# Custom loss function
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import CategoricalCrossentropy
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

class CustomLossLL2(tf.keras.losses.Loss):
    def __init__(self, lambda_t, model, mu, sigma_TL):
        super().__init__()
        self.lambda_t = lambda_t
        self.cross_entropy = CategoricalCrossentropy()
        self.model = model
        self.mu = tf.convert_to_tensor(mu, dtype=tf.float32)
        self.sigma_TL = tf.convert_to_tensor(sigma_TL, dtype=tf.float32)

    def call(self, y_true, y_pred):
        ce_loss = self.cross_entropy(y_true, y_pred)
        wt = self.get_weights_from_model()
        reg_term = self.regularization_term(wt)

        return ce_loss + (self.lambda_t * tf.linalg.matmul(reg_term, wt))

    def get_weights_from_model(self):
        model_weights = []
        for layer in self.model.layers:
            if len(layer.get_weights()) > 0:
                model_weights.append(layer.get_weights()[0])
        # return tf.concat([tf.reshape(w, [-1]) for w in model_weights], axis=0)
        return model_weights

    def regularization_term(self, wt):
        diff = wt - self.mu
        reg_term = 0.5 * tf.linalg.matmul(tf.linalg.matmul(tf.linalg.inv(self.sigma_TL), diff[0]), tf.transpose(diff[0]))
        reg_term += 0.5 * tf.math.log(tf.linalg.det(self.sigma_TL))
        return reg_term


# Custom training loop
def custom_train_step(model, optimizer, x, y, custom_loss):
    with tf.GradientTape() as tape:
        y_pred = model(x, training=True) # Perform a forward pass and compute the predictions
        loss = custom_loss(y, y_pred) # Compute the custom loss
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    return loss


def train_weight_LL2(X_train, y_train, lambd, mu, sigma_TL, num_tier=10, learning_rate = 0.01):
    n_classes = np.unique(y_train).size
    y_one_hot = tf.keras.utils.to_categorical(y_train, num_classes=n_classes)

    lambda_t = lambd  # Regularization parameter

    model = Sequential([
        Dense(n_classes, input_shape=(X_train.shape[1],), activation='softmax')  # Adjust input_shape to match the number of features in X
    ])

    # Compile the model
    optimizer = Adam(learning_rate)
    custom_loss = CustomLossLL2(lambda_t, model, mu, sigma_TL)

    # Custom training loop
    epochs = num_tier
    lowest_loss = float('inf')
    best_weights = None
    best_model = None

    for epoch in range(epochs):
        loss = custom_train_step(model, optimizer, X_train, y_one_hot, custom_loss)
        loss_value = loss.numpy()
        if epoch % 20 == 0:
            print(f"Epoch {epoch}, Loss: {loss.numpy()}")

        if (abs(loss_value) < lowest_loss):
            lowest_loss = abs(loss_value)
            best_model = model
            best_weights = [layer.get_weights() for layer in model.layers]

    return best_model, best_weights, lowest_loss

def GetConfusionMatrix(model, X_train, X_test, y_train, y_test):
    y_pred_prob = model.predict(X_train)
    y_pred = np.argmax(y_pred_prob, axis=1)

    print("Classification TRAIN DATA \n=======================")
    print(classification_report(y_true= y_train, y_pred=y_pred))
    print("Confusion matrix \n=======================")
    print(confusion_matrix(y_true= y_train, y_pred=y_pred))

    y_pred_prob = model.predict(X_test)
    y_pred = np.argmax(y_pred_prob, axis=1)
    print("Classification TEST DATA \n=======================")
    print(classification_report(y_true=y_test, y_pred=y_pred))
    print("Confusion matrix \n=======================")
    print(confusion_matrix(y_true=y_test, y_pred=y_pred))


def tgt_test_wLTL(data, target_subjects ,condition):
        tgt_data = target_subjects + "_test"

        if condition == "noEA":
            X = data[target_subjects]['Raw_csp']
            y = data[target_subjects]['Raw_csp_label']
            X_test = data[tgt_data]['Raw_csp']
            y_test = data[tgt_data]['Raw_csp_label']
            store_ws = 'wt_Raw'

        else:
            X = data[target_subjects]['EA_csp']
            y = data[target_subjects]['EA_csp_label']
            X_test = data[tgt_data]['EA_csp']
            y_test = data[tgt_data]['EA_csp_label']
            store_ws = 'wt_EA'

        mu = data[target_subjects]['mu_ws']
        sigma_TL = data[target_subjects]['Sigma_TL']

        X_train = X
        y_train = y
        
        model, weights, loss = train_weight_LL2(X_train=X_train, y_train=y_train, mu =mu, sigma_TL=sigma_TL, lambd= 0.1, num_tier=5000, learning_rate= 0.01)
        print("weights of ", str(target_subjects), ": ", weights)
        print("loss of ", str(target_subjects), ": ", loss)
        data[target_subjects][store_ws] = weights

        GetConfusionMatrix(model, X_train, X_test, y_train, y_test)

tgt_test_wLTL(CSP2D_Epoch, target_subjects= target_data_0 ,condition = condition_wLTL)

Epoch 0, Loss: -2.2522635459899902
Epoch 20, Loss: -2.8747618198394775
Epoch 40, Loss: -3.5457820892333984
Epoch 60, Loss: -4.41274356842041
Epoch 80, Loss: -5.433821201324463
Epoch 100, Loss: -6.564009666442871
Epoch 120, Loss: -7.735199928283691
Epoch 140, Loss: -8.889394760131836
Epoch 160, Loss: -9.982175827026367
Epoch 180, Loss: -10.9820556640625
Epoch 200, Loss: -11.869194030761719
Epoch 220, Loss: -12.633204460144043
Epoch 240, Loss: -13.270782470703125
Epoch 260, Loss: -13.783778190612793
Epoch 280, Loss: -14.17738151550293
Epoch 300, Loss: -14.45861530303955
Epoch 320, Loss: -14.635427474975586
Epoch 340, Loss: -14.715901374816895
Epoch 360, Loss: -14.707903861999512
Epoch 380, Loss: -14.618825912475586
Epoch 400, Loss: -14.455578804016113
Epoch 420, Loss: -14.2245512008667
Epoch 440, Loss: -13.931619644165039
Epoch 460, Loss: -13.582201957702637
Epoch 480, Loss: -13.181341171264648
Epoch 500, Loss: -12.733694076538086
Epoch 520, Loss: -12.243658065795898
Epoch 540, Loss: -11